In [35]:
import requests
from urllib.parse import quote
import pandas as pd
from bs4 import BeautifulSoup

In [36]:
url = 'http://book.interpark.com/display/collectlist.do?_method=bestsellerHourNew&bookblockname=b_gnb&booklinkname=%BA%A3%BD%BA%C6%AE%C1%B8&bid1=w_bgnb&bid2=LiveRanking&bid3=main&bid4=001'
req =  requests.get(url)
html = req.text



In [37]:
soup = BeautifulSoup(html, 'html.parser')

In [38]:
bse = soup.select('div.rankBestContentList > ol > li')

In [39]:
bs = bse[11]


In [40]:
#순위
#rank = bs.select_one('.rankNumber.digit2').find('span')['class'][1][-1:]
rank = bs.select('.rankNumber.digit2>span')
if len(rank) == 1 :
    rank = bs.select_one('.rankNumber.digit2').find('span')['class'][1][-1:]
else:
    rank = bs.select_one('.rankNumber.digit2').find('span')['class'][1][-1:] + bs.select_one('.rankNumber.digit2').find_all('span')[1]['class'][1][-1:]
rank

'12'

In [41]:
#title
title = bs.select_one('div.itemName').get_text()
title

'투자는 디테일에 있다 '

In [42]:
#가격
price = bs.select_one('div.priceWrap').get_text().split('\n')[2:4]
price = int(price[0].replace(',',''))
price


16200

In [43]:
bs = bse[0]
href = bs.find('a')['href'][69:78]
href

'354069935'

In [44]:
sub_url = f'http://book.interpark.com/product/BookDisplay.do?_method=detail&sc.shopNo=0000400000&sc.prdNo={href}&sc.saNo=003003001&bid1=Best_zone&bid2=LiveRanking&bid3=PRD&bid4=001'
sub_url

'http://book.interpark.com/product/BookDisplay.do?_method=detail&sc.shopNo=0000400000&sc.prdNo=354069935&sc.saNo=003003001&bid1=Best_zone&bid2=LiveRanking&bid3=PRD&bid4=001'

In [45]:
sub_req =  requests.get(sub_url)
sub_html = sub_req.text
sub_soup = BeautifulSoup(sub_html, 'html.parser')


In [46]:
#저자
writer = sub_soup.select_one('div.bookInfoBox').find('ul').get_text().split('\n')[1][4:]
writer


'김난도, 전미영, 최지혜, 이향은, 이준영'

In [47]:
#출판사
company = sub_soup.select_one('div.bookInfoBox').find('ul').get_text().split('\n')[2][6:]
company

'미래의창'

In [52]:
# 발행일
year = sub_soup.select_one('div.bookInfoBox').get_text().split('\n')[4][5:]
year

'2021년 04월 28일'

In [49]:
# 쪽수
page = sub_soup.select_one('div.bookInfoBox').get_text().split('\n')[-4][5:]
page

'452'

In [50]:
lines = []
for bs in bse:
    href = bs.find('a')['href'][69:78]
    rank = bs.select('.rankNumber.digit2>span')
    if len(rank) == 1 :
        rank = bs.select_one('.rankNumber.digit2').find('span')['class'][1][-1:]
    else:
        rank = bs.select_one('.rankNumber.digit2').find('span')['class'][1][-1:] + bs.select_one('.rankNumber.digit2').find_all('span')[1]['class'][1][-1:]
    title = bs.select_one('div.itemName').get_text()
    price = bs.select_one('div.priceWrap').get_text().split('\n')[2:4]
    price = int(price[0].replace(',',''))
    price

    sub_url = f'http://book.interpark.com/product/BookDisplay.do?_method=detail&sc.shopNo=0000400000&sc.prdNo={href}&sc.saNo=003003001&bid1=Best_zone&bid2=LiveRanking&bid3=PRD&bid4=001'
    sub_req =  requests.get(sub_url)
    sub_html = sub_req.text
    sub_soup = BeautifulSoup(sub_html, 'html.parser')
    writer = sub_soup.select_one('div.bookInfoBox').find('ul').get_text().split('\n')[1][4:]
    company = sub_soup.select_one('div.bookInfoBox').find('ul').get_text().split('\n')[2][6:]
    year = sub_soup.select_one('div.bookInfoBox').get_text().split('\n')[4][5:]
    page = sub_soup.select_one('div.bookInfoBox').get_text().split('\n')[-4][5:]

    lines.append([rank,title,price,writer,company,year,page])
    

In [59]:
df = pd.DataFrame(lines, columns=['순위','제목','가격','저자','출판사','발행일','쪽수'])
df

,순위,제목,가격,저자,출판사,발행일,쪽수
0,1,트렌드 코리아 2022,16200,"김난도, 전미영, 최지혜, 이향은, 이준영",미래의창,2021년 10월 06일,452
1,2,2022 현직교사들이 들려주는 면접레시피,42300,"류은진, 양왕경, 이광한, 이지혜, 양왕경, 이광한, 이지혜",미래가치,2021년 11월 12일,1104
2,3,아직 오지 않은 날들을 위하여,14400,파스칼 브뤼크네르역 : 이세진,인플루엔셜,2021년 11월 12일,320
3,4,거인의 포트폴리오,16200,강환국,페이지2북스,2021년 11월 08일,404
4,5,불편한 편의점(15만부 기념 윈터 에디션),12600,김호연,나무옆의자,2021년 04월 20일,268
5,6,거꾸로 읽는 세계사,15750,유시민,돌베개,2021년 10월 29일,404
6,6,달러구트 꿈 백화점,12420,이미예,팩토리나인,2020년 07월 08일,300
7,8,작은 별이지만 빛나고 있어(10만 부 기념 한정판 에디션),13500,소윤,북로망스,2021년 09월 29일,274
8,9,달러구트 꿈 백화점. 2,12420,이미예,팩토리나인,2021년 07월 27일,308
9,10,오십에 읽는 논어,14400,최종엽,유노북스,2021년 11월 03일,300


In [60]:
df.to_csv('인터파크_베스트셀러(211207).csv', index = False)